In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
train = pd.read_csv('../input/train.csv')

In [ ]:
train.head()

# Exploratory Data Analysis


## Missing Data


In [ ]:
sns.heatmap(train.isnull(),yticklabels=False,cbar=False,cmap='viridis')

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',data=train,palette='RdBu_r')

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',hue='Sex',data=train,palette='RdBu_r')

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',hue='Pclass',data=train,palette='rainbow')

In [ ]:
sns.distplot(train['Age'].dropna(),kde=False,color='darkred',bins=30)

In [ ]:
train['Age'].hist(bins=30,color='darkred',alpha=0.7)

In [ ]:
sns.countplot(x='SibSp',data=train)

In [ ]:
train['Fare'].hist(color='green',bins=40,figsize=(8,4))

In [ ]:
import cufflinks as cf
cf.go_offline()

In [ ]:
train['Fare'].iplot(kind='hist',bins=30,color='green')

___
# Data Cleaning


In [ ]:
plt.figure(figsize=(12, 7))
sns.boxplot(x='Pclass',y='Age',data=train,palette='winter')

In [ ]:
def impute_age(cols):
    Age = cols[0]
    Pclass = cols[1]
    
    if pd.isnull(Age):

        if Pclass == 1:
            return 37

        elif Pclass == 2:
            return 29

        else:
            return 24

    else:
        return Age

In [ ]:
train['Age'] = train[['Age','Pclass']].apply(impute_age,axis=1)

Now let's check that heat map again!

In [ ]:
sns.heatmap(train.isnull(),yticklabels=False,cbar=False,cmap='viridis')

In [ ]:
train.drop('Cabin',axis=1,inplace=True)

In [ ]:
train.head()

In [ ]:
train.dropna(inplace=True)

## Converting Categorical Features 


In [ ]:
train.info()

In [ ]:
sex = pd.get_dummies(train['Sex'],drop_first=True)
embark = pd.get_dummies(train['Embarked'],drop_first=True)

In [ ]:
train.drop(['Sex','Embarked','Name','Ticket'],axis=1,inplace=True)

In [ ]:
train = pd.concat([train,sex,embark],axis=1)

In [ ]:
train.head()



# Building a Logistic Regression model



In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train.drop('Survived',axis=1), train['Survived'], test_size=0.30, random_state=101)

## Training and Predicting

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)

In [ ]:
test = pd.read_csv('../input/test.csv')

In [ ]:
test.drop('Cabin',axis=1,inplace=True)

In [ ]:
test['Age'] = test[['Age','Pclass']].apply(impute_age,axis=1)

In [ ]:
test['Fare'].fillna((test['Fare'].mean()), inplace=True)

In [ ]:
sex = pd.get_dummies(test['Sex'],drop_first=True)
embark = pd.get_dummies(test['Embarked'],drop_first=True)

In [ ]:
test.drop(['Sex','Embarked','Name','Ticket'],axis=1,inplace=True)

In [ ]:
test= pd.concat([test,sex,embark],axis=1)

In [ ]:
predictions = logmodel.predict(X_test)

Let's move on to evaluate our model!

## Evaluation

We can check precision,recall,f1-score using classification report!

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test,predictions))

In [ ]:
test_pred = logmodel.predict(test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfmodel = RandomForestClassifier(n_estimators=300)

In [ ]:
rfmodel.fit(X_train, y_train)

In [ ]:
pred_rf = rfmodel.predict(X_test)

In [ ]:
print(classification_report(pred_rf, y_test))

In [ ]:
rfmodel.score(X_test, y_test)

In [ ]:
pred_results = rfmodel.predict(test)

In [ ]:
test['Survived'] = pred_results

In [ ]:
submission = test[['PassengerId', 'Survived']]

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)

In [ ]:
knn.fit(X_train, y_train)

In [ ]:
pred = knn.predict(X_test)


In [ ]:
print(classification_report(y_test, pred))


In [ ]:
error_rate = []

for i in range(1,40):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, y_train)
    pred_i = knn.predict(X_test)
    error_rate.append(np.mean(pred_i != y_test))

In [ ]:
plt.figure(figsize=[10,6])
plt.plot(range(1,40),error_rate,color='blue', linestyle='dashed', marker='o',markerfacecolor='red', markersize=10)
plt.title('Error Rate vs. K Value')
plt.xlabel('K')
plt.ylabel('Error Rate')

In [ ]:

knn = KNeighborsClassifier(n_neighbors=25)
knn.fit(X_train, y_train)
pred = knn.predict(X_test)

In [ ]:
print(classification_report(y_test, pred))


In [ ]:
#catboost
train = pd.read_csv('../input/train.csv')
test =  pd.read_csv('../input/test.csv')

In [ ]:
train.fillna(-999,inplace=True)
test.fillna(-999,inplace=True)

In [ ]:
x = train.drop('Survived',axis=1)
y = train.Survived

In [ ]:
x.dtypes

In [ ]:
cat_features_index = np.where(x.dtypes != float)[0]

In [ ]:
import hyperopt
from catboost import Pool, CatBoostClassifier, cv

In [ ]:
model = CatBoostClassifier(eval_metric='Accuracy',use_best_model=True,random_seed=42)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=101)

In [ ]:
model.fit(X_train,y_train,cat_features=cat_features_index,eval_set=(X_test,y_test))

In [ ]:
from sklearn.metrics import accuracy_score
print('the test accuracy is :{:.6f}'.format(accuracy_score(y_test,model.predict(X_test))))

In [ ]:
pred = model.predict(test)
pred = pred.astype(np.int)
submission = pd.DataFrame({'PassengerId':test['PassengerId'],'Survived':pred})

In [ ]:
submission.to_csv('catboost.csv',index=False)
